In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv("Police_Incidents.csv")
print(df.head)

C:\Users\kgvin\AppData\Local\Temp\ipykernel_4848\3307952256.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Police_Incidents.csv")


<bound method NDFrame.head of         Incident Number w/year  Year of Incident Service Number ID Watch  \
0                  203058-2022              2022    203058-2022-01     1   
1                  264509-2016              2016    264509-2016-01     2   
2                  004110-2015              2015    004110-2015-01     2   
3                  022647-2023              2023    022647-2023-01     1   
4                  112585-2015              2015    112585-2015-01     3   
...                        ...               ...               ...   ...   
1127417            276915-2016              2016    276915-2016-01     2   
1127418            163009-2017              2017    163009-2017-01     1   
1127419            265586-2015              2015    265586-2015-01     3   
1127420            192459-2015              2015    192459-2015-01     1   
1127421            205093-2019              2019    205093-2019-01     2   

                     Call (911) Problem  \
0             

# Add Zipcode-Percentage column

In [2]:
# Calculate the total incident count for each Zip Code
zipcode_counts = df.groupby('Zip Code').size().reset_index(name='IncidentCount')

# Calculate the total number of incidents in the subset
total_incidents = zipcode_counts['IncidentCount'].sum()

# Calculate the percentage for each Zip Code and round to 2 decimal places
zipcode_counts['Percentage'] = round((zipcode_counts['IncidentCount'] / total_incidents) * 100, 2)

# Convert the DataFrame to a dictionary
zipcode_incident_dict = dict(zip(zipcode_counts['Zip Code'], zipcode_counts['Percentage']))

# Sort the dictionary by percentage in descending order
sorted_zipcode_incident_dict = dict(sorted(zipcode_incident_dict.items(), key=lambda item: item[1], reverse=True))

# Create a DataFrame from the sorted dictionary
table_df = pd.DataFrame(sorted_zipcode_incident_dict.items(), columns=['Zip Code', 'Percentage'])

# Merge the 'Percentage' column from 'table_df' into the original 'df' and rename it
df = df.merge(table_df[['Zip Code', 'Percentage']], on='Zip Code', how='left').rename(columns={'Percentage': 'Zipcode-Percentage'})

# Print the updated DataFrame to verify
print(df)

        Incident Number w/year  Year of Incident Service Number ID Watch  \
0                  203058-2022              2022    203058-2022-01     1   
1                  264509-2016              2016    264509-2016-01     2   
2                  004110-2015              2015    004110-2015-01     2   
3                  022647-2023              2023    022647-2023-01     1   
4                  112585-2015              2015    112585-2015-01     3   
...                        ...               ...               ...   ...   
1127417            276915-2016              2016    276915-2016-01     2   
1127418            163009-2017              2017    163009-2017-01     1   
1127419            265586-2015              2015    265586-2015-01     3   
1127420            192459-2015              2015    192459-2015-01     1   
1127421            205093-2019              2019    205093-2019-01     2   

                     Call (911) Problem  \
0                        PSE/09 - THEFT   
1

In [3]:
print(df['Zipcode-Percentage'])

0          3.29
1          2.20
2          3.37
3          0.88
4          2.28
           ... 
1127417    0.36
1127418    3.68
1127419    0.97
1127420    4.81
1127421    1.96
Name: Zipcode-Percentage, Length: 1127422, dtype: float64


# Add Zipcode-Day-Percentage column

In [4]:
# Calculate the total incident count for each Zip Code and Day
zip_day_counts = df.groupby(['Zip Code', 'Day1 of the Week']).size().reset_index(name='IncidentCount')

# Calculate the total incident count for each Zip Code
zip_total_counts = df.groupby('Zip Code').size().reset_index(name='TotalIncidentCount')

# Merge the two DataFrames to get the percentage of crimes for each Zip Code and Day
result_df = pd.merge(zip_day_counts, zip_total_counts, on='Zip Code')
result_df['Percentage'] = (result_df['IncidentCount'] / result_df['TotalIncidentCount']) * 100

# Pivot the DataFrame to create a table of Zip Codes and Days with Percentage of Crimes
crime_percentage_table = result_df.pivot(index='Zip Code', columns='Day1 of the Week', values='Percentage').fillna(0)

# Round the percentages to 2 decimal places
crime_percentage_table = crime_percentage_table.round(2)

# Reshape the DataFrame to have only two columns (Zip Code and Day)
crime_percentage_table = crime_percentage_table.reset_index().melt(id_vars='Zip Code', var_name='Day', value_name='Percentage')

# Filter out rows with 0 percentages
crime_percentage_table = crime_percentage_table[crime_percentage_table['Percentage'] > 0]

# Convert 'Zip Code' and 'Day1 of the Week' columns to the same data type in both DataFrames
df['Zip Code'] = df['Zip Code'].astype(str)
crime_percentage_table['Zip Code'] = crime_percentage_table['Zip Code'].astype(str)

# Merge the 'crime_percentage_table' into the original DataFrame 'df'
df = df.merge(crime_percentage_table, left_on=['Zip Code', 'Day1 of the Week'], right_on=['Zip Code', 'Day'], how='left')

# Rename the 'Percentage' column to 'Zipcode-Day-Percentage'
df.rename(columns={'Percentage': 'Zipcode-Day-Percentage'}, inplace=True)

# Print the updated DataFrame to verify
print(df['Zipcode-Day-Percentage'])

# print(crime_percentage_table)


0          14.13
1          15.33
2          14.17
3          14.23
4          14.38
           ...  
1127417    14.06
1127418    14.24
1127419    15.99
1127420    13.73
1127421    13.67
Name: Zipcode-Day-Percentage, Length: 1127422, dtype: float64


# Add Zipcode-Time-Percentage Column

In [5]:
# Define time bins
time_bins = {
    'Night': ('18:00', '23:59'),
    'Early Morning': ('00:00', '05:59'),
    'Morning': ('06:00', '11:59'),
    'Afternoon': ('12:00', '17:59')
}

# Create a function to assign time bins
def assign_time_bin(time):
    for bin_label, (start_time, end_time) in time_bins.items():
        if start_time <= time <= end_time:
            return bin_label
    return 'Unknown'

# Assign time bins to each row in the DataFrame
df['Time Bin'] = df['Time1 of Occurrence'].apply(assign_time_bin)

# Calculate the total incident count for each Zip Code and Time Bin
zip_time_counts = df.groupby(['Zip Code', 'Time Bin']).size().reset_index(name='IncidentCount')

# Calculate the total incident count for each Zip Code
zip_total_counts = df.groupby('Zip Code').size().reset_index(name='TotalIncidentCount')

# Merge the two DataFrames to get the percentage of crimes for each Zip Code and Time Bin
result_df = pd.merge(zip_time_counts, zip_total_counts, on='Zip Code')
result_df['Percentage'] = (result_df['IncidentCount'] / result_df['TotalIncidentCount']) * 100

# Create a DataFrame with 'Zip Code', 'Time Bin', and 'Percentage' columns
crime_percentage_table = result_df[['Zip Code', 'Time Bin', 'Percentage']]

# Round the percentages to 2 decimal places
crime_percentage_table = crime_percentage_table.round(2)

# Merge the 'crime_percentage_table' into the original DataFrame 'df'
df = df.merge(crime_percentage_table, left_on=['Zip Code', 'Time Bin'], right_on=['Zip Code', 'Time Bin'], how='left')

# Rename the 'Percentage' column to 'Zip-Time-Percentage'
df.rename(columns={'Percentage': 'Zip-Time-Percentage'}, inplace=True)

# Print the updated DataFrame to verify
print(df)


        Incident Number w/year  Year of Incident Service Number ID Watch  \
0                  203058-2022              2022    203058-2022-01     1   
1                  264509-2016              2016    264509-2016-01     2   
2                  004110-2015              2015    004110-2015-01     2   
3                  022647-2023              2023    022647-2023-01     1   
4                  112585-2015              2015    112585-2015-01     3   
...                        ...               ...               ...   ...   
1127417            276915-2016              2016    276915-2016-01     2   
1127418            163009-2017              2017    163009-2017-01     1   
1127419            265586-2015              2015    265586-2015-01     3   
1127420            192459-2015              2015    192459-2015-01     1   
1127421            205093-2019              2019    205093-2019-01     2   

                     Call (911) Problem  \
0                        PSE/09 - THEFT   
1

In [6]:
print(df['Zip-Time-Percentage'])

0          33.66
1          30.19
2          29.00
3          17.14
4          27.10
           ...  
1127417    19.19
1127418    34.35
1127419    47.23
1127420    33.47
1127421    27.39
Name: Zip-Time-Percentage, Length: 1127422, dtype: float64


In [11]:
import csv
input_file = 'C:/Users/kgvin/Downloads/res.csv'

# Initialize an empty dictionary
data_dict = {}

# Open the CSV file and read it
with open(input_file, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    # Skip the header row if it exists
    next(csv_reader, None)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        if len(row) == 2:
            key, value = row[0].strip(), row[1].strip()
            data_dict[key] = value
# Create a new column based on the incident type values
df['Type of Incident'] = df['Type of Incident'].str.strip()  # Strip spaces from 'Type of Incident' in DataFrame
df['Incident_Score'] = df['Type of Incident'].map(lambda x: data_dict.get(x.strip(), None))


# Now, your DataFrame 'df' has a new column 'Incident_Value' containing the values based on the incident type.


# Save a new CSV file 

In [12]:
# Create a new DataFrame with the specified columns
new_df = df[['Zip Code', 'Day1 of the Week', 'Time1 of Occurrence', 'Time Bin', 'Division', 'Sector', 'Zip-Time-Percentage', 'Zipcode-Day-Percentage', 'Zipcode-Percentage', 'Incident_Score', 'Type of Incident' ]]

# Save the new DataFrame to a CSV file
new_df.to_csv('filtered_data_2.csv', index=False)
